In [ ]:
# ! pip install sounddevice 

In [1]:
import torch
import sounddevice as sd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import scipy.signal

# Carrega o modelo e o processador
model_id = "freds0/distil-whisper-large-v3-ptbr"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.eval()

c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nielsen.castelo\.cache\huggingface\hub\models--freds0--distil-whisper-large-v3-ptbr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate 

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [4]:
# ! pip install faster-whisper


In [2]:
from faster_whisper import WhisperModel

model = WhisperModel("medium")

c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nielsen.castelo\.cache\huggingface\hub\models--Systran--faster-whisper-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from faster_whisper import WhisperModel
import sounddevice as sd
import numpy as np
import tempfile
import scipy.io.wavfile
import os

# === Parâmetros de gravação ===
DURATION = 5  # segundos
SAMPLE_RATE = 16000

print("🎙️ Gravando áudio...")
audio = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
sd.wait()
print("✅ Gravação concluída.")

# === Conversão do áudio para int16 ===
audio = np.squeeze(audio)
audio = np.clip(audio, -1.0, 1.0)  # evita estouros na conversão
audio_int16 = (audio * 32767).astype(np.int16)

# === Salva como arquivo temporário WAV ===
with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
    scipy.io.wavfile.write(tmp.name, SAMPLE_RATE, audio_int16)
    audio_path = tmp.name

# === Carrega o modelo Whisper ===
model = WhisperModel("medium", compute_type="float32", device="cpu")  # Use "cuda" se tiver GPU

# === Transcrição ===
segments, info = model.transcribe(audio_path, language="pt")

# === Exibe a transcrição ===
print(f"\n📝 Transcrição (idioma detectado: {info.language})")for segment in segments:
    print(f"[{segment.start:.2f}s - {segment.end:.2f}s] {segment.text}")

# (Opcional) Remove o arquivo temporário
os.remove(audio_path)


🎙️ Gravando áudio...
✅ Gravação concluída.

📝 Transcrição (idioma detectado: pt)
[0.00s - 5.00s]  Alô, alô comigo, faça a transcrição do...
